In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [33]:
import math

import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from util.plots import scatter_plot
from util.data import load_mnist, encode_data
from typing import List, Dict
from tqdm.notebook import tqdm
from algorithms import SaccadeGenerator, GridCells, SpatialPooler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

<IPython.core.display.Javascript object>

### Algorithm

1. movement to update location layer
    - calculate new phase - input: displacement vector and old phase; applies translation by rotation and scaling + mod 1 --> outputs binary vector A_t,move^loc
    - if no movement is given, activate random location ???
2. predict sensory information from location layer output
    - predictions represented by pi_t^in = activity of distal dendritic segments - Eq2 in paper
        --> modularoty effect
3. calculate activity in sensory layer
    - all cells share same feedforward receptive fields --> pretrained CNN ouputs (128,) feature vector and each column is mapped to on feature
    - sensory features represented by sparse subset of mini-columns = W_t^in
    - if cell is predicted and in active mini-columns: activate cell and inhibit other cells in mini-column; else: all cells become active
    - - multiple cells can be active if predicted by location layer
    - formula uq: 6 for activation logic
4. update location layer based on sensory cues
    - location layer receives input from sensory layer (in form of W_t^in???)
    - calculate overlap EQ 3 & 4
    - repeat because new movement is received

Learning:
- strenghtening between connections of both layer
- when new object is learned: location representation is randomly initialised (each object has its own location space = multiple objects can be represented at the same time because their representation has low probability of overlap
- sensory layer: random cell in each column will be active
- form reciprocal connections on one of their dendritic segments (d') (EQ 5 & 6)
--> if synapse exist it is unaffected otherwise it is formed

### Setup Data and Saccade Generator

In [23]:
(x_train, y_train), (x_test, y_test) = load_mnist()
x_train, x_test = encode_data(x_train, x_test)

<IPython.core.display.Javascript object>

In [24]:
saccade_size = (10, 10)

<IPython.core.display.Javascript object>

In [79]:
saccade_generator_train = SaccadeGenerator(
    x=x_train,
    y=y_train,
    saccade_size=saccade_size,
    no_saccades=5,
)

<IPython.core.display.Javascript object>

In [80]:
saccade_generator_test = SaccadeGenerator(
    x=x_test,
    y=y_test,
    saccade_size=saccade_size,
    no_saccades=5,
)

<IPython.core.display.Javascript object>

### Train Spatial Pooler

In [56]:
sp = SpatialPooler(
    input_dimension=saccade_size,
    column_dimension=(100,),
    connection_sparsity=0.5,
    permanence_threshold=0.5,
    stimulus_threshold=5,
    permanence_increment=0.1,
    permanence_decrement=0.02,
    column_sparsity=0.02,
    potential_pool_radius=8,
    boost_strength=10,
)

<IPython.core.display.Javascript object>

In [57]:
def get_active_columns(saccade_generator, learn):
    active_columns = np.zeros(shape=(x_train.shape[0], 100))

    for idx, obj in enumerate(tqdm(saccade_generator)):
        active_columns_overlap = np.zeros(shape=(100,))

        for saccade in obj:
            feature = saccade["feature"].reshape((10, 10))
            wc = sp.compute(feature, learn=True)
            active_columns_overlap[wc] = 1

        active_columns[idx] = active_columns_overlap.copy()

    return active_columns

<IPython.core.display.Javascript object>

In [58]:
train_columns = get_active_columns(saccade_generator_train, learn=True)
# test_columns = get_active_columns(saccade_generator_test, learn=False)

  0%|          | 0/60000 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

### Evaluate Classification Performance of Spatial Pooler

In [59]:
parameters = {"C": [0.1, 0.5, 0.7, 1.0], "penalty": ["none", "l1", "l2"]}
lr = LogisticRegression()

clf = GridSearchCV(lr, parameters, n_jobs=-1)
clf.fit(train_columns, y_train)
clf.best_score_

/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesi

/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1)

/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/moritzeich/.pyenv/versions/3.10.0/envs/bachelor-thesis-research/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alter

0.10566666666666666

<IPython.core.display.Javascript object>

In [ ]:
sp.save_state("experiments/spatial_pooler")

### Setup GridCells - Sensory Location Layers

In [82]:
no_features = math.prod(saccade_size)

<IPython.core.display.Javascript object>

In [83]:
gc = GridCells(
    no_modules=4,
    grid_cell_module_dimension=(16, 16),
    orientation=60,
    scale=20,
    no_columns=no_features,
    no_cells_per_column=8,
    location_layer_threshold=6,
    seed=42,
)

<IPython.core.display.Javascript object>

### Learn Objects

In [ ]:
for obj in saccade_generator_train:
    gc.learn_object(obj)

    break